In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn import linear_model
import spacy
from pure_sklearn.map import convert_estimator
import pickle

In [ ]:
nlp  = spacy.load('nl_core_news_lg') 

In [ ]:
data = pd.read_pickle('./data/processed/woorden_met_hetofde.pickle')
data.head()

In [ ]:
data.det.value_counts()

In [ ]:
16675-5137

In [ ]:
# Separate majority and minority classes
df_majority = data[data.det=='de']
df_minority = data[data.det=='het']
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=16675,    # to match majority class
                                 random_state=123) # reproducible results

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# Display new class counts
df_upsampled.det.value_counts()

In [ ]:
df_upsampled.head()

In [ ]:
X = df_upsampled.woord_vec.tolist()
y = df_upsampled.det.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### KNN 

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3,n_jobs=-1)

In [ ]:
%%timeit
neigh.fit(X_train, y_train)

In [ ]:
predicted = neigh.predict(X_test)

In [ ]:
np.mean(predicted==y_test)

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
predicted = clf.predict(X_test)

In [ ]:
np.mean(predicted==y_test)

In [ ]:

# # convert to pure python estimator
# clf_pure_predict = convert_estimator(clf)
# with open("./data/dumps/random_forest_clf.pkl", "wb") as f:
#     pickle.dump(clf_pure_predict, f)

In [ ]:
validation_data = [(token.text, token.vector) for token in doc if token.pos_ == "NOUN"]


In [ ]:
vector_list = [v[1] for v in validation_data]
word_list   = [v[0] for v in validation_data]

In [ ]:
# load pickled model
with open("./data/dumps/random_forest_clf.pkl", "rb") as f:
    clf = pickle.load(f)

In [ ]:
# make prediction with pure-predict object
predictions = clf.predict_proba(vector_list)

In [ ]:
json_result_list = []
for prediction, word in zip(predictions,word_list):
    json_result = {}
    json_result['woord'] = word
    json_result['probability'] = {'de' : prediction[0] , 'het' :   prediction[1]}
    json_result_list.append(json_result)

In [ ]:
json_result_list

## MLP 

In [ ]:
clf = MLPClassifier(random_state=1, learning_rate='adaptive',
                    max_iter=200).fit(X_train, y_train)

In [ ]:
import pickle
# convert to pure python estimator
clf_pure_predict = convert_estimator(clf)
with open("./data/dumps/mlp_classifier.pickle","wb") as f:
    pickle.dump(clf_pure_predict, f)

In [ ]:
predicted = clf.predict(X_test)

In [ ]:
np.mean(predicted==y_test)

## SGD

In [ ]:
%%timeit
clf = linear_model.SGDClassifier().fit(X_train, y_train)

In [ ]:
%%timeit
predicted = clf.predict(X_test)

In [ ]:
%%timeit
np.mean(predicted==y_test)